In [ ]:
import pandas as pd
import re
import requests
# from google.colab import files

In [ ]:
subscriberDF = pd.read_csv('../data/Original_Subscriber_Information.csv')
subscriberDF

In [ ]:
#function to get rid of 0s and add decimal
def transform_value(value):
    if pd.isnull(value):
        return value
    value_str = str(int(value))
    if value_str.endswith('0000'):
        value_int = int(value / 10000)
        return float(value_int / 100)
    else:
        return value

In [ ]:
#apply to DF
subscriberDF['Purchase Amount'] = subscriberDF['Purchase Amount'].apply(transform_value)
subscriberDF

In [ ]:
#fetch exchange rates
def fetch_exchange_rates(api_key, currencies):
    currency_list = ','.join(currencies)
    url = f'https://openexchangerates.org/api/latest.json?app_id={api_key}&symbols={currency_list}'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()['rates']
    else:
        raise ValueError(f"Failed to fetch exchange rates. Status code: {response.status_code}")

api_key = '418dd91a76a743c7bee44a1feb77fe51'

currencies = [
    'AED', 'AUD', 'BGN', 'BRL', 'CAD', 'CHF', 'CLP', 'COP', 'CRC', 'CZK',
    'DKK', 'EGP', 'EUR', 'GBP', 'GHS', 'HKD', 'HUF', 'IDR', 'ILS', 'INR',
    'JPY', 'KRW', 'KZT', 'LBP', 'MXN', 'MYR', 'NOK', 'NZD', 'PEN', 'PHP',
    'PLN', 'QAR', 'RON', 'RSD', 'RUB', 'SAR', 'SEK', 'SGD', 'THB', 'TRY',
    'UAH', 'VND', 'ZAR'
]

exchange_rates = fetch_exchange_rates(api_key, currencies)
print(exchange_rates)

In [ ]:
#convert to USD
def convert_to_usd(amount, currency, exchange_rates):
    if currency in exchange_rates:
        return amount / exchange_rates[currency]
    else:
        return amount

def convert_currencies_to_usd(data_frame, amount_column, currency_column, exchange_rates):
    data_frame[amount_column] = data_frame.apply(lambda row: convert_to_usd(row[amount_column], row[currency_column], exchange_rates), axis=1)
    return data_frame

In [ ]:
subscriberDF = convert_currencies_to_usd(subscriberDF, 'Purchase Amount', 'Currency', exchange_rates)
subscriberDF

In [ ]:
appDF = pd.read_csv('../data/Original_App_activity.csv')

print(appDF.nunique())
# print unique sesion types
print(appDF['App Session Platform'].unique())
print(appDF['App Activity Type'].unique())

print("shape0: ", appDF.shape)
# drop na
appDF.dropna()
print("shape1: ", appDF.shape)
#drop nan
appDF = appDF.dropna(subset=['App Activity Type', 'App Session Platform', 'App Session Date'])
print("shape2: ", appDF.shape)
print(appDF['App Activity Type'].unique())
print(appDF['App Session Platform'].unique())

print(appDF.nunique())

appDF.to_csv('../data/app_nonull.csv', index=False)

In [ ]:
# Use currencies to determine the country of the subscriber
currency_to_country = {
    'AED': 'United Arab Emirates',
    'AUD': 'Australia',
    'BGN': 'Bulgaria',
    'BRL': 'Brazil',
    'CAD': 'Canada',
    'CHF': 'Switzerland',
    'CLP': 'Chile',
    'COP': 'Colombia',
    'CRC': 'Costa Rica',
    'CZK': 'Czech Republic',
    'DKK': 'Denmark',
    'EGP': 'Egypt',
    'EUR': 'European Union',
    'GBP': 'United Kingdom',
    'GHS': 'Ghana',
    'HKD': 'Hong Kong',
    'HUF': 'Hungary',
    'IDR': 'Indonesia',
    'ILS': 'Israel',
    'INR': 'India',
    'JPY': 'Japan',
    'KRW': 'South Korea',
    'KZT': 'Kazakhstan',
    'LBP': 'Lebanon',
    'MXN': 'Mexico',
    'MYR': 'Malaysia',
    'NOK': 'Norway',
    'NZD': 'New Zealand',
    'PEN': 'Peru',
    'PHP': 'Philippines',
    'PLN': 'Poland',
    'QAR': 'Qatar',
    'RON': 'Romania',
    'RSD': 'Serbia',
    'RUB': 'Russia',
    'SAR': 'Saudi Arabia',
    'SEK': 'Sweden',
    'SGD': 'Singapore',
    'THB': 'Thailand',
    'TRY': 'Turkey',
    'UAH': 'Ukraine',
    'USD': 'United States',
    'VND': 'Vietnam',
    'ZAR': 'South Africa'
}

#skip rows with na or nan in currency
subscriberDF['Country'] = subscriberDF['Currency'].apply(lambda currency: currency_to_country[currency] if currency in currency_to_country else None)

subscriberDF.nunique()

In [ ]:
subscriberDF.to_csv('../data/subscriber_USD.csv', index=False)

merged_df = subscriberDF.merge(appDF, on='ID', how='left')


In [ ]:
# count unique values in each column
origSubscriberDF = pd.read_csv('../data/Original_Subscriber_Information.csv')
print("orig:   ", origSubscriberDF.shape)
print("clean:  ", subscriberDF.shape)
print("merged: ", merged_df.shape)
print("orig:")
print(origSubscriberDF.nunique())
print("cleaned:")
print(subscriberDF.nunique())
print("merged:")
print(merged_df.nunique())

In [ ]:

# merged_df.to_csv("merged.csv")
# files.download("merged.csv")

merged_df.to_csv("../data/merged_A.csv")